# The data verification & validation process

In [1]:
import pandas as pd
import plotly.express as px
import plotly.io as pio   
pio.kaleido.scope.mathjax = None

## Importing and refining our dataset

In [2]:
# Import data
cols = ["Year Code", "ID", "Sex", "Age", "Date of death", "DATE OF RECEPTION", "DATE OF RETURN FOR SEPULTURE", "DATE OF BURIAL OR CREMATION", "PLACE OF DEATH", "PLACE OF DEATH CATEGORY CODE"]
data = pd.read_csv("data/donor_register_april17.csv", low_memory=True, usecols=cols)
data.head()

,Year Code,ID,Sex,Age,Date of death,DATE OF RECEPTION,DATE OF RETURN FOR SEPULTURE,DATE OF BURIAL OR CREMATION,PLACE OF DEATH,PLACE OF DEATH CATEGORY CODE
0,1883,1.0,M,71.0,8/5/1883,11/5/1883,13/6/1883,13/6/1883,Callan Park Hospital,100
1,1883,2.0,M,78.0,13/5/1883,15/5/1883,19/5/1883,19/5/1883,Sydney Hospital,23
2,1883,3.0,M,38.0,22/5/1883,24/5/1883,16/10/1883,16/10/1883,Gladesville Mental Hospital,3
3,1883,4.0,M,60.0,7/7/1883,08/07/1883,16/10/1883,16/10/1883,Sydney Hospital,23
4,1883,5.0,M,52.0,10/7/1883,13/7/1883,16/10/1883,16/10/1883,Sydney Hospital,23


In [3]:
# Rename columns
cols = ["year", "id", "sex", "age", "death_date", "reception_date", "return_date_sepulture", "burial_date", "death_place", "place_code"]
data.columns = cols

data.head()

,year,id,sex,age,death_date,reception_date,return_date_sepulture,burial_date,death_place,place_code
0,1883,1.0,M,71.0,8/5/1883,11/5/1883,13/6/1883,13/6/1883,Callan Park Hospital,100
1,1883,2.0,M,78.0,13/5/1883,15/5/1883,19/5/1883,19/5/1883,Sydney Hospital,23
2,1883,3.0,M,38.0,22/5/1883,24/5/1883,16/10/1883,16/10/1883,Gladesville Mental Hospital,3
3,1883,4.0,M,60.0,7/7/1883,08/07/1883,16/10/1883,16/10/1883,Sydney Hospital,23
4,1883,5.0,M,52.0,10/7/1883,13/7/1883,16/10/1883,16/10/1883,Sydney Hospital,23


### Basic preliminary data analysis

In [4]:
count_by_year = data.groupby(["year"], as_index=False).size()
count_by_year.columns = ["year", "count"]

summary_stats = count_by_year["count"].describe()

fig = px.line(x=count_by_year["year"], y=count_by_year["count"])

fig.update_layout(
    title="Number of entries in Anatomy Registers by year",
    xaxis_title="Year",
    yaxis_title="Count",
    showlegend=False,
    # margin=dict(l=30, r=30, t=30, b=30)
)

# with open("out/data_by_year.pdf", "wb") as f:
#     f.write(fig.to_image(format="pdf"))

fig.show()

print(count_by_year["count"].describe())

count    101.000000
mean      75.336634
std       37.972695
min       10.000000
25%       50.000000
50%       69.000000
75%      105.000000
max      164.000000
Name: count, dtype: float64


In [5]:
grouped = data.groupby(["year", "sex"], as_index=False, observed=True).size()
fig = px.bar(grouped, x="year", y="size", color="sex", )
fig.update_layout(
    title="Anatomy donors over time by sex",
    xaxis_title="Year",
    yaxis_title="Count",
    # margin=dict(l=20, r=20, t=20, b=20)
)

# with open("out/data_by_sex.pdf", "wb") as f:
#     f.write(fig.to_image(format="pdf"))

fig.show()

## Ensuring data completeness

In [6]:
# Drop empty rows
data = data.dropna(how="all")

### Investigating missing values

In [7]:
data.shape

(7609, 10)

In [8]:
data.isna().sum()

year                        0
id                          0
sex                         5
age                        10
death_date               2436
reception_date              0
return_date_sepulture    1680
burial_date              4408
death_place                 6
place_code                  0
dtype: int64

As we cannot infer the ages or biological sex of the patients in the anatomy registers, we will simply remove the entries with missing values for these attributes.

In [9]:
data = data[~data["sex"].isna() & ~data["age"].isna()]

As this project aims to perform a geographical analysis on the dataset, we will also choose to remove the 6 entries which are missing a place of death.

In [10]:
data = data[~data["death_place"].isna()]

### Imputing missing values

We currently have about 2400 entries in the dataset which are missing values for `death_date`.

One way to overcome this is to fill in the missing `death_date` values with the values from `reception_date`. We can do this because bodies were usually received into the custody of the university 2 to 3 days after the death of the patient, so the reception date of a body was always on or after the patient's date of death.

In [11]:
data["death_date"] = data["death_date"].fillna(data["reception_date"])

In [12]:
data.isna().sum()

year                        0
id                          0
sex                         0
age                         0
death_date                  0
reception_date              0
return_date_sepulture    1673
burial_date              4394
death_place                 0
place_code                  0
dtype: int64

In [13]:
data.shape

(7588, 10)

In the end we still have many missing values for `return_date_sepulture` and `burial_date`, however these should not impact our investigation too much for this research project.

## Ensuring data consistency 

When our program first imported the dataset, it tried to automatically detect what datatype each column was. The result of this automatic detection often clues us into potential data validation issues in the dataset; for example, if a column which should only hold integers gets parsed as a floating point number, chances are there are non-integer values present in the attribute. Let's see how our dataset was parsed.

In [14]:
data.dtypes

year                       int64
id                       float64
sex                       object
age                      float64
death_date                object
reception_date            object
return_date_sepulture     object
burial_date               object
death_place               object
place_code                 int64
dtype: object

There are a few things to note here:
- `id` and `age` should be integers, and we should convert `id` into the index of our dataframe
- `sex` and `place_code` can be parsed as categorical variables
- The 4 date attributes should be stored as `datetime` objects

### Converting attributes to integer types

Let's first check why `age`, and `id` were parsed as floats rather than integers. Are there any non-integer values in these attributes?

In [15]:
def can_convert_to_int(value):
    if isinstance(value, float):
        return value.is_integer()
    try:
        int(value)
        return True
    except ValueError:
        return False

In [16]:
data[~data["id"].apply(can_convert_to_int)]

,year,id,sex,age,death_date,reception_date,return_date_sepulture,burial_date,death_place,place_code
2044,1920,2044.5,M,0.0,18/6/1920,18/6/1920,14/1/1921,NaN,SOUTH SYDNEY WOMEN'S HOSPITAL,70
3333,1938,3330.5,M,0.0,8/12/1938,8/12/1938,[blank],NaN,STILLBORN; Premature twin males received from ...,70
4050,1949,4046.5,M,0.0,13/7/1949,13/7/1949,NaN,7/6/1950,Briarways Private Hospital Guildford,70


It seems there are three entries in the dataset which have non-integer values for `id`. Fixing these would require editing the id's for thousands of entries; instead, it is simpler and cleaner to just remove these rows. We then subsequently perform the datatype conversion and set the attribute as our index.

In [17]:
# Selecting only those rows which have integer id values
data = data[data["id"].apply(can_convert_to_int)]

# Performing the data type conversion
data["id"] = data.loc[:, "id"].astype("int")

# Setting id as the index of the dataset
data.set_index("id", inplace=True)

Now we check `age`.

In [18]:
data[~data["age"].apply(can_convert_to_int)]

,year,sex,age,death_date,reception_date,return_date_sepulture,burial_date,death_place,place_code
id,,,,,,,,,


There are no entries in our dataset with non-integer values for `age`. The error, therefore, must have been in the original dataset (we may have dropped the erroneous rows in the data completeness section above). Let's update these datatypes.

In [19]:
data["year"] = data["year"].astype("int")
data["age"] = data["age"].astype("int")

### Converting attributes to categoricals

Let's first convert the `sex` attribute to the `pd.Categorical` datatype. Simultaneously, we can validate that the dataset only contains two categories: `M` and `F`.

In [20]:
data["sex"] = data["sex"].astype("category")
data["sex"].unique()

['M', 'F']
Categories (2, object): ['F', 'M']

However, before we convert the `place_code` attribute, we should check why it was parsed in as floating point values in the first place. We will use the same function as above for this.

In [21]:
data[~data["place_code"].apply(can_convert_to_int)]

,year,sex,age,death_date,reception_date,return_date_sepulture,burial_date,death_place,place_code
id,,,,,,,,,


The column does not seem to contain any floating point values. To ensure our categories are properly defined, though, we should first convert these values to `int`s and then `Categorical`s.

In [22]:
data["place_code"] = data["place_code"].astype("int")
data["place_code"] = data["place_code"].astype("category")
data["place_code"].unique()

[100, 23, 3, 11, 44, ..., 22, 8, 9, 7, 2]
Length: 22
Categories (22, int64): [1, 2, 3, 5, ..., 44, 70, 100, 101]

### Converting attributes to datetime types

#### Running regex verification

Before we attempt to cast the values in the date columns, we must ensure the values do not contain any erroneous strings. This can cause problems in the casting process.

In [23]:
# Using regex to check for any values which contain more than just a date or a date out of range
regex = "^(0?[0-9]|[12]\d|3[01])\/(0?[0-9]|1[0-2])\/(000\d|00\d{2}|0\d{3}|1\d{3}|2000)$"
sentinel = "00/00/0000"
data_copy = data.fillna({"death_date": sentinel, "reception_date": sentinel, "return_date_sepulture": sentinel, "burial_date": sentinel})

In [24]:
data_copy[~data_copy["death_date"].str.match(regex)].iloc[:, :7]

,year,sex,age,death_date,reception_date,return_date_sepulture,burial_date
id,,,,,,,
1898,1919,M,81,25/1/2020,27/1/1919,8/7/1919,00/00/0000
5014,1961,M,76,26/87/1961,30/8/1961,00/00/0000,13/9/1962
5044,1961,F,80,20/11/9161,20/11/1961,00/00/0000,9/11/1962
7116,1977,F,90,5/101977,5/10/1977,18/10/1978,00/00/0000


In [25]:
data_copy[~data_copy["reception_date"].str.match(regex)].iloc[:, :7]


,year,sex,age,death_date,reception_date,return_date_sepulture,burial_date
id,,,,,,,
1727,1917,M,70,15/11/1917,missing,16/3/1918,00/00/0000
2408,1925,M,83,22/4/1925,23 /4/1925,31/7/1925,00/00/0000


In [26]:
data_copy[~data_copy["return_date_sepulture"].str.match(regex)].iloc[:, :7]

,year,sex,age,death_date,reception_date,return_date_sepulture,burial_date
id,,,,,,,
2432,1925,M,83,15/8/1925,18/8/1925,3 /12/1925,00/00/0000
2644,1929,F,0,20/3/1929,20/3/1929,crossed out 27/9/1929,00/00/0000
3176,1936,F,0,31/8/1936,31/8/1936,[blank],00/00/0000
3179,1936,F,0,3/9/1936,3/9/1936,[blank],00/00/0000
3181,1936,F,0,15/9/1936,15/9/1936,[blank],00/00/0000
3182,1936,F,0,15/9/1936,15/9/1936,[blank],00/00/0000
3184,1936,F,0,21/9/1936,21/9/1936,[blank],00/00/0000
3185,1936,F,0,21/9/1936,21/9/1936,[blank],00/00/0000
3203,1937,M,0,16/4/1937,16/4/1937,[blank],00/00/0000


In [27]:

data_copy[~data_copy["burial_date"].str.match(regex)].iloc[:, :7]

,year,sex,age,death_date,reception_date,return_date_sepulture,burial_date
id,,,,,,,
6726,1974,M,86,9/4/1974,9/4/1974,13/3/1975,4/4//1975


As you can see, there are small data entry errors in some rows which would have prevented all 4 columns from being parsed properly. Let's fix these before performing the data type conversion.

In [28]:
# death_date
data.loc[1898, "death_date"] = "25/1/1919" # checked manually
data.loc[5014, "death_date"] = "26/8/1961" # checked manually
data.loc[5044, "death_date"] = "20/11/1961" # checked manually
data.loc[7116, "death_date"] = "5/10/1977" 

# reception_date
data.loc[1727, "reception_date"] = pd.NA
data.loc[2408, "reception_date"] = "23/4/1925" 

# return_date_sepulture
data.loc[2432, "return_date_sepulture"] = "3/12/1925" 
data.loc[2644, "return_date_sepulture"] = "27/09/1929" 
data = data.replace(["[blank]", "-"], pd.NA)

# burial_date
data.loc[6726, "burial_date"] = "4/4/1975"

#### Some more niche errors

Now let's retry converting our dates.

In [29]:
try:
    data["death_date"] = pd.to_datetime(data["death_date"], format="%d/%m/%Y")
except Exception as e:
    print(e)

day is out of range for month, at position 5405. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.


Now we are getting into the territory of more complex errors which manual searching or regular expression methods cannot pick up. It seems that at position 5405 in the dataframe there is an invalid date.

In [30]:
data.iloc[5405]

year                                                                  1964
sex                                                                      M
age                                                                     78
death_date                                                       31/6/1964
reception_date                                                   31/5/1964
return_date_sepulture                                            18/5/1965
burial_date                                                      18/5/1965
death_place              Lisieux CORR PRIVATE Hospital, Backhouse St, W...
place_code                                                               1
Name: 5424, dtype: object

Indeed, June has but 30 days, so it is impossible for someone to have a recorded date of death as `31/06/1964`. Through looking at the `reception_date` we can infer that the entry is a typo and the death date was really meant to be `31/05/1964`.

In [31]:
data.iloc[5405, 3] = "31/05/1964"

Finally, let's convert all our 4 columns to datetime format.

In [32]:
data["death_date"] = pd.to_datetime(data["death_date"], format="%d/%m/%Y")
data["reception_date"] = pd.to_datetime(data["reception_date"], format="%d/%m/%Y")
data["return_date_sepulture"] = pd.to_datetime(data["return_date_sepulture"], format="%d/%m/%Y")
data["burial_date"] = pd.to_datetime(data["burial_date"], format="%d/%m/%Y")

In [33]:
data.dtypes

year                              int64
sex                            category
age                               int64
death_date               datetime64[ns]
reception_date           datetime64[ns]
return_date_sepulture    datetime64[ns]
burial_date              datetime64[ns]
death_place                      object
place_code                     category
dtype: object

### Standardising place of death

The `place_of_death` attribute spans numerous locations, including private addresses, aged care homes, private and public hospitals, public asylums, mental asylums, morgues, and more. As the records were all written and transcribed by hand, many of the errors you'd expect to find with manually annotated data are present in this column, for example, two different places of death which differ only by punctuation or spelling. This section deals with the standardisation of this part of the dataset.

The main goal here is to clean up the existing entries into $n$ unique locations without any duplicates. That is, we want to combine all near duplicates of a location into one categorical value. So, for instance, we would want to combine the two entries 

- Community Hospital Long Jetty formerly 7 Gosford Rd The Entrance
- Community Hospital Wyong Road Long Jetty

into just "Community Hospital Long Jetty".

Along the way we will also perform categorisation of the `place_of_death` entries so we can more easily analyse the spread of locations.

#### Basic cleaning

Let's deal with the basic cleaning first:
- Standardising capitalisation of all entries using title case
- Removing all punctuation from the locations
- Deleting any surrounding whitespace from the entries
- Deleting any extra whitespace between words (each word should be delimited by exactly one space)

In [34]:
data["death_place"] = data["death_place"].str.lower()
data["death_place"] = data["death_place"].str.replace(r'[!\"#$%&\'()*+,-./:;<=>?@\[\\\]^_`{|}~]', "", regex=True)
data["death_place"] = data["death_place"].str.strip()

In [35]:
def ensure_single_space(string: str):
    return " ".join(string.split())

data["death_place"] = data["death_place"].apply(ensure_single_space)

#### Trimming extra information


As can be seen in the case above, some entries contain extra information which makes the similarity checking more inaccurate, such as the inclusion of where the site "formerly" existed. If we want to automate the de-duplication of the dataset, we will first have to remove these phrases from the entries. Below is a list of the phrases we will treat.

- "formerly"
- "dead on arrival"
- "died at"
- Compound-valued place of death conjoined through either "and" or "then"

In [36]:
# Remove any occurrences of "formerly" as well as the text following it
data["death_place"] = data["death_place"].str.replace(r"\s?formerly.*$", "", regex=True)

# Remove any occurrences of "dead on arrival"
data["death_place"] = data["death_place"].str.replace(r"\s?dead on arrival\s?", "", regex=True)

# Remove any occurrences of "died at" at the start of the string
data["death_place"] = data["death_place"].str.replace(r"^(died at\s?)", "", regex=True)

When dealing with conjoined places of death, any record that has ‘morgue’ in it should be attributed to the morgue, but otherwise the first entry should be regarded as place of death.

In [37]:
def clean_conjoined(loc: str):
    if " and " in loc:
        loc_split = loc.split(" and ")
        if "morgue" in loc_split[1]:
            return loc_split[1]
        else:
            return loc_split[0]
    
    if " then " in loc:
        loc_split = loc.split(" then ")
        if "morgue" in loc_split[1]:
            return loc_split[1]
        else:
            return loc_split[0]
    
    return loc

data["death_place"] = data["death_place"].apply(clean_conjoined)

#### Categorising locations

In this section, we aim to categorise entries based on the type of institution found in the `place_of_death` column. Below is the new key we will use to categorise entries. Note that some of the codes are reliant on the pre-existing `place_code` attribute.

| Category             | Classification criteria                                                                                                                |
|----------------------|----------------------------------------------------------------------------------------------------------------------------------------|
| Private residence    | A private address (commencing with a street, flat, lot or unit number OR "at home") |
| Aged care            | Locations containing "Nursing", "Convalescent", "Aged Care", "Village", "Lodge", "Home", "Aged"  + some known aged care institutions   |
| Hospice              | Locations containing "Hospice" + some known hospices                                                                                   |
| Private hospital     | Locations containing "Private Hospital" or "surgery"                                                                                   |
| Public hospital      | All hospitals without "private" in their name, including district hospitals                                                            |
| Public asylum        | Currently coded 12 + three special cases (Liverpool, George St, Rookwood)                                                              |
| Public mental asylum | Currently coded 13 + locations containing "mental asylum" or "psychiatric"                                                             |
| Aged 2 and under     | Currently coded 70                                                                                                                     |
| Morgues              | Currently coded 6 + locations containing "morgue"                                                                                      |
| Other                | Other                                                                                                                                  |

In [38]:
data = data.sort_values(by="death_place")
data["death_place_category"] = pd.NA

##### Private residences

In [39]:
# If the location starts with "flat", "unit", "lot", or contains "at home", it is a private residence
private_residence_pattern = r"^(?:flat|unit|lot)|\bat home\b"

# If the location starts with a digit, and also contains an address suffix, it is a private residence
address_keywords = ["avenue", "av", "ave", "boulevard", "bvd", "chase", "ch", "circuit", "cct", "close", "cl", "court", "ct", "crescent", "cr", "cres", "crest", "crst", "drive", "dr", "esplanade", "esp", "grange", "gra", "grove", "gr", "highway", "hwy", "lane", "la", "mall", "parade", "pde", "parkway", "pwy", "place", "pl", "road", "rd", "square", "sq", "street", "st", "terrace", "tce", "way", "walk"]
address_pattern = fr"^\d.*\b(?:{'|'.join(address_keywords)})\b"

data.loc[data["death_place"].str.contains(private_residence_pattern), "death_place_category"] = "private residence"
data.loc[data["death_place"].str.contains(address_pattern) & data['death_place_category'].isna(), "death_place_category"] = "private residence"

##### Aged care

In [40]:
aged_care_keywords = ["nursing home", "convalescent", "aged care", "village", "lodge", "aged", "homes", "war veterans home"]
aged_care_pattern = "|".join(fr"\b{word}\b" for word in aged_care_keywords)

query = data["death_place"].str.contains(aged_care_pattern) & data["death_place_category"].isna()
data.loc[query, "death_place_category"] = "aged care"

##### Hospice

In [41]:
data.loc[data["death_place"].str.contains("hospice") & data["death_place_category"].isna(), "death_place_category"] = "hospice"
data.loc[data["death_place"].str.contains("home of peace") & data["death_place_category"].isna(), "death_place_category"] = "hospice"
data.loc[data["death_place"].str.contains("little sisters of the poor") & data["death_place_category"].isna(), "death_place_category"] = "hospice"

##### Private hospitals

In [42]:
private_hospital_pattern = r"\bprivate\b.*\bhospital\b|surgery"
data.loc[data["death_place"].str.contains(private_hospital_pattern) & data["death_place_category"].isna(), "death_place_category"] = "private hospital"

##### Public hospitals

In [43]:
# Public hospitals are any leftover entries which contain "hospital" 
data.loc[data["death_place"].str.contains(r"\bhospital\b") & data["death_place_category"].isna(), "death_place_category"] = "public hospital"

##### Public asylums

In [44]:
# Public asylums are all those currently coded 12
data.loc[data["place_code"] == 12, "death_place_category"] = "public asylum"

# Three special cases
data.loc[data["death_place"].str.contains(r"\b(?:liverpool\sasylum|george\sstreet\sasylum|rookwood\sasylum)\b") & data["death_place_category"].isna(), "death_place_category"] = "public asylum"


##### Public mental asylums

In [45]:
# Public mental asylums are currently coded 13
data.loc[data["place_code"] == 13, "death_place_category"] = "public mental asylum"

# Include any entries containing "mental asylum"
data.loc[data["death_place"].str.contains(r"\bmental\sasylum\b") & data["death_place_category"].isna(), "death_place_category"] = "public mental asylum"

# Include any entries containing "psychiatric"
data.loc[data["death_place"].str.contains(r"\bpsychiatric\b") & data["death_place_category"].isna(), "death_place_category"] = "public mental asylum"


##### Aged 2 and under

In [46]:
# Patients who died under the age of 2 are currently coded 70
data.loc[data["place_code"] == 70, "death_place_category"] = "aged 2 or under"

##### Morgues

In [47]:
# Patients who died in morgues are currently coded 6
data.loc[data["place_code"] == 6, "death_place_category"] = "morgue"

# Also include any leftover entries containing "morgue"
data.loc[data["death_place"].str.contains(r"\bmorgue\b") & data["death_place_category"].isna(), "death_place_category"] = "morgue"

##### Other

In [48]:
# Fill in all remaining entries as "other"
data["death_place_category"] = data["death_place_category"].fillna("other")


##### Categorising left-over "dirty" data 

Now that we have categorised our data to the best of our ability using keywords, let's examine what didn't get categorised (i.e. was categorised as "other"). This will show us what our algorithm above didn't pick up.

In [49]:
other = data[data["death_place_category"] == "other"][["death_place", "death_place_category"]]
other.to_csv("out/category_other.csv")

On manual examination of the remaining entries, we can pick up a few trends:
- Remaining entries beginning with a house number did not get picked up as `private residence` due to spelling mistakes and unusual or missing street names.
- There are many records which are comprised of only a street name and a suburbl, or even just a suburb e.g. `albany st east gosford`, `blue mountains`. These should all be identified as `private residence`.

## Ensuring data validity

Now that our dataset is **consistent** i.e. all the entries in each attribute are of the same datatype and format, and do not contain any duplicates, we can move onto checking whether the values in each attribute actually make sense in the context of the dataset.

Note that `sex` has already been validated above.

### `year`

The dataset should range from the year 1883 to 1984.

In [50]:
print("min: ", data["year"].min())
print("max: ", data["year"].max())

min:  1883
max:  1983


### `age`

The ages in the dataset should range from 0 to about 115.

In [51]:
print("min: ", data["age"].min())
print("max: ", data["age"].max())

min:  0
max:  110


### The 4 `date` columns

- The `death_date` and `reception_date` columns should range between 1883 and 1983 (there is no hard limit on the values for the other two death columns as the embalming period of bodies ranged between 12-18 months).
- The difference between the `death_date` and `reception_date` of an entry should be no longer than 1 month.
- The date of reception should be equal to or greater than the date of death.
- For those entries which have values for `return_date_sepulture` and `burial_date`, the burial date should be equal to or greater than the return date.

#### Checking date range

In [52]:
date_cols = ["death_date", "reception_date", "return_date_sepulture", "burial_date"]

In [53]:
# Checking date range
for col in date_cols[:2]:
    if data[col].dt.year.min() < 1883: 
        print(f"min year in {col} < 1883")
    if data[col].dt.year.max() > 1983:
        print(f"max year in {col} > 1983")

max year in death_date > 1983


There is at least one entry in the dataset which has a death_death greater than 1984.

In [54]:
data[data["death_date"].dt.year > 1983]

,year,sex,age,death_date,reception_date,return_date_sepulture,burial_date,death_place,place_code,death_place_category
id,,,,,,,,,,
339,1898,M,83,1989-09-10,1898-09-10,1898-11-07,NaT,rookwood asylum,15,public asylum
7567,1983,M,80,1984-03-25,1983-03-28,NaT,1985-07-02,ryde hospital denistone rd eastwood,1,public hospital


In [55]:
# Fix the typos
data.loc[339, "death_date"] = pd.to_datetime("1898-09-10")
data.loc[7567, "death_date"] = pd.to_datetime("1983-03-25")

#### Checking for nearby dates

In [56]:
data[(data["reception_date"] - data["death_date"]).dt.days > 10][["death_date", "reception_date"]].sort_values(by="death_date")

,death_date,reception_date
id,,
638,1905-03-31,1905-04-13
635,1905-03-31,1905-05-01
1560,1913-03-26,1916-03-27
1292,1913-06-07,1913-08-08
2042,1920-06-12,1920-06-25
2243,1922-09-22,1922-10-23
2449,1924-11-02,1925-11-04
4947,1961-05-01,1961-06-02
5139,1962-07-04,1962-07-25


Upon checking the physical registers to confirm the true dates, a few errors were found around entry `638`. These are fixed below, along with other entries.

In [57]:
# Fixing entries around 638
data.loc[635, "reception_date"] = pd.to_datetime("01/04/1905", dayfirst=True)
data.loc[636, "death_date"] = pd.to_datetime("03/04/1905", dayfirst=True)
data.loc[637, "death_date"] = pd.to_datetime("06/04/1905", dayfirst=True)
data.loc[638, "death_date"] = pd.to_datetime("12/04/1905", dayfirst=True)

data.loc[823, "reception_date"] = pd.to_datetime("21/4/1908", dayfirst=True)
data.loc[1292, "reception_date"] = pd.to_datetime("08/06/1913", dayfirst=True)
data.loc[1560, "death_date"] = pd.to_datetime("26/03/1916", dayfirst=True)
data.loc[2042, "reception_date"] = pd.to_datetime("15/06/1920", dayfirst=True)
data.loc[2243, "death_date"] = pd.to_datetime("22/10/1922", dayfirst=True)
data.loc[2449, "death_date"] = pd.to_datetime("02/11/1925", dayfirst=True)
data.loc[4947, "reception_date"] = pd.to_datetime("02/05/1961", dayfirst=True)
data.loc[5139, "reception_date"] = pd.to_datetime("05/07/1962", dayfirst=True)
# 5210: the dates are correct
data.loc[5436, "death_date"] = pd.to_datetime("14/06/1964", dayfirst=True)
data.loc[5503, "death_date"] = pd.to_datetime("05/11/1964", dayfirst=True)
data.loc[6020, "death_date"] = pd.to_datetime("14/07/1968", dayfirst=True)
data.loc[6235, "death_date"] = pd.to_datetime("03/04/1970", dayfirst=True)
data.loc[6290, "death_date"] = pd.to_datetime("13/08/1970", dayfirst=True)
data.loc[6471, "death_date"] = pd.to_datetime("14/12/1971", dayfirst=True)
data.loc[6574, "death_date"] = pd.to_datetime("22/09/1972", dayfirst=True)
data.loc[6770, "death_date"] = pd.to_datetime("27/07/1974", dayfirst=True)
data.loc[6873, "reception_date"] = pd.to_datetime("14/07/1975", dayfirst=True)
# 7316: the dates are correct, and due to the considerable delay in receipt the body was unsuitable for use

In [58]:
data[(data["burial_date"] - data["return_date_sepulture"]).dt.days > 30][["return_date_sepulture", "burial_date"]].sort_values(by="burial_date")

,return_date_sepulture,burial_date
id,,
5340,1964-04-04,1964-06-04
5628,1966-10-30,1966-11-30
5780,1966-10-31,1967-10-31
5775,1968-05-16,1968-06-16
7390,1982-02-08,1982-03-19
7345,1981-06-25,1982-06-29
7476,1983-05-23,1983-07-15
7528,1984-08-30,1984-10-04


Due to the bodies having been embalmed, it is possible for the difference between the return date and burial date to be greater than one month. Also, delays often occured when the body was returned back to the family for burial.

In [59]:
data.loc[5340, "burial_date"] = pd.to_datetime("04/04/1964", dayfirst=True)
# 5628: dates are correct, the delay was due to the body being buried
# 5775: 
data.loc[5780, "return_date_sepulture"] = pd.to_datetime("21/10/1967", dayfirst=True)
data.loc[7345, "return_date_sepulture"] = pd.to_datetime("25/6/1982", dayfirst=True)
# 7390: dates are correct 
# 7476: dates are correct 
# 7528: dates are correct


#### Checking for sequential dates

Note that as the data was annotated and collected by hand, small variations between the pairs of dates are not uncommon – in these cases we will simply fix the dates to ensure `death_date` <= `reception_date` and `return_date_sepulture` <= `burial_date`.

In [60]:
data[data["death_date"] > data["reception_date"]][["death_date", "reception_date"]].sort_values(by="death_date")

,death_date,reception_date
id,,
600,1904-07-20,1904-06-22
647,1905-05-26,1905-04-27
839,1908-06-12,1908-06-11
964,1910-03-30,1910-03-21
1283,1913-05-21,1913-05-20
1617,1916-08-19,1916-08-10
1631,1917-02-10,1917-01-11
1689,1917-07-08,1917-07-05
1877,1919-10-24,1918-10-25


In [61]:
# Note: when there is doubt in these cases, the reception_date is assumed as correct.
data.loc[600, "death_date"] = pd.to_datetime("1904-06-20")
data.loc[647, "death_date"] = pd.to_datetime("1905-04-26")
data.loc[839, "death_date"] = pd.to_datetime("1908-06-11")
data.loc[964, "reception_date"] = pd.to_datetime("1910-03-31")
data.loc[1283, "death_date"] = pd.to_datetime("1913-05-20")
data.loc[1617, "death_date"] = pd.to_datetime("1916-08-09")
data.loc[1631, "death_date"] = pd.to_datetime("1917-01-10")
data.loc[1689, "reception_date"] = pd.to_datetime("1917-07-09")
data.loc[1877, "death_date"] = pd.to_datetime("1918-10-24")
data.loc[2004, "death_date"] = pd.to_datetime("1920-01-24")
data.loc[2286, "death_date"] = pd.to_datetime("1923-06-18")
data.loc[2686, "reception_date"] = pd.to_datetime("1929-09-12")
data.loc[3760, "death_date"] = pd.to_datetime("1943-12-14")
data.loc[3762, "death_date"] = pd.to_datetime("1943-12-18")
data.loc[4988, "death_date"] = pd.to_datetime("1961-07-05")
data.loc[5036, "reception_date"] = pd.to_datetime("1961-10-23")
data.loc[5278, "death_date"] = pd.to_datetime("1963-05-28")
data.loc[6067, "death_date"] = pd.to_datetime("1968-11-11")
data.loc[6079, "reception_date"] = pd.to_datetime("1968-12-13")
data.loc[6082, "death_date"] = pd.to_datetime("1968-12-27")
data.loc[6722, "death_date"] = pd.to_datetime("1974-03-26")
data.loc[6996, "reception_date"] = pd.to_datetime("1976-09-08")
data.loc[7278, "death_date"] = pd.to_datetime("1979-06-04")
data.loc[7567, "death_date"] = pd.to_datetime("1983-03-25")

In [62]:
# Checking we have fixed all errors
data[data["death_date"] > data["reception_date"]][["death_date", "reception_date"]].sort_values(by="death_date")

,death_date,reception_date
id,,


In [63]:
data[data["return_date_sepulture"] > data["burial_date"]][["return_date_sepulture", "burial_date"]]

,return_date_sepulture,burial_date
id,,
5550,1966-05-16,1966-05-15
5516,1966-05-23,1965-05-24


In [64]:
# These are just small typos
data.loc[5550, "return_date_sepulture"] = pd.to_datetime("1966-05-15")
data.loc[5516, "return_date_sepulture"] = pd.to_datetime("1965-05-15")

In [65]:
# Checking we have fixed all errors
data[data["return_date_sepulture"] > data["burial_date"]][["return_date_sepulture", "burial_date"]]

,return_date_sepulture,burial_date
id,,


# Exporting the cleaned dataset

Now that the data validation process is complete, we can export the final dataset and use it for analysis.

In [66]:
data.sort_index().to_csv("out/donors.csv")